In [ ]:
# default_exp pde

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
import numpy as np
from nangs.bocos import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# PDE

> This module contains the PDE class with the basic functionality to solve PDEs with NNs.

In [ ]:
#export

from nangs.utils import *

class PDE:
    "PDE class with basic functionality to solve PDEs with NNs"
    def __init__(self, inputs, outputs, params=None):
        
        self.input_keys = inputs
        self.output_keys = outputs
        self.param_keys = params
        
        # initialize values
        self.train_inputs = initListOfStr(self.input_keys)
        self.test_inputs = initListOfStr(self.input_keys)
        self.outputs = initListOfStr(self.output_keys)
        self.params = None
        if self.param_keys:
            self.params = initListOfStr(self.param_keys)
            
        # make sure there are not repeated keys
        checkNoRepeated(self.input_keys, self.output_keys)
        if self.params:
            checkNoRepeated(self.input_keys, self.param_keys)
            checkNoRepeated(self.param_keys, self.output_keys)
        
        self.bocos = []

    def summary(self):
        "Print a summary of the PDE inputs, outputs, params and bocos."
        print('inputs (train): ', {name: values for name, values in zip(self.input_keys, self.train_inputs)})
        print('inputs (test): ', {name: values for name, values in zip(self.input_keys, self.test_inputs)})
        print('outputs: ', {name: values for name, values in zip(self.output_keys, self.outputs)})
        print('params: ', {name: values for name, values in zip(self.param_keys, self.params)})
        print('bocos: ', [boco.type for boco in self.bocos])
        print('')
        
    def setValues(self, values, train=True):
        "Set values for inputs and params"
        checkValidDict(values)
        for key in values:
            value = values[key]
            if key in self.input_keys: 
                if train: 
                    setValue(self.input_keys, self.train_inputs, key, value)
                else: 
                    setValue(self.input_keys, self.test_inputs, key, value)
            elif key in self.param_keys: 
                if train: 
                    setValue(self.param_keys, self.params, key, value)    
                else: 
                    raise Exception('You cannot set params in test data !')
            elif key in self.output_keys:
                raise Exception('You cannot set values to outputs !')
            else:
                raise Exception('Key '+ key +' not found !')
                
    def addBoco(self, boco):
        "Add a boco to the list of bocos"
        boco.addBoco(self.input_keys, self.output_keys)
        self.bocos += [boco]
        
    def bocoSummary(self):
        "Print summary of each boco"
        for boco in self.bocos: 
            boco.summary(self.input_keys, self.output_keys, self.param_keys) 


## Instantiate a PDE

In [ ]:
pde = PDE(inputs=['x', 't'], outputs=['p'], params=['u'])

*inputs*, *outputs* and *params* (which are optionals) must be lists of strings with non-repeated elements or you will get an error.

In [ ]:
try:
    pde = PDE(inputs=['x', 't'], outputs=['x'], params=['u'])
except Exception as e:
    assert str(e) == "Repeated item x", "assertion failed"

In [ ]:
#hide

pde = PDE(['a'], ['b'])
pde = PDE(inputs=['a'], outputs=['b'])
pde = PDE(inputs=['a', 'b', 'c'], outputs=['d'])
pde = PDE(inputs=['a'], outputs=['b', 'c', 'd'])
pde = PDE(inputs=['a'], outputs=['b'], params=['c'])
pde = PDE(inputs=['a', 'b'], outputs=['c'], params=['d', 'e', 'f'])

try:
    pde = PDE(inputs=['a'])
except Exception as e:
    assert str(e) == "__init__() missing 1 required positional argument: 'outputs'", "assertion failed"
    
try:
    pde = PDE(outputs=['a'])
except Exception as e:
    assert str(e) == "__init__() missing 1 required positional argument: 'inputs'", "assertion failed"
    
try:
    pde = PDE(inputs=['a'], outputs=42)
except Exception as e:
    assert str(e) == "42 must be a list of strings", "assertion failed"
    
try:
    pde = PDE(inputs=None, outputs=42)
except Exception as e:
    assert str(e) == "None must be a list of strings", "assertion failed"
    
try:
    pde = PDE(inputs=['a', 42], outputs=['b'])
except Exception as e:
    assert str(e) == "42 must be a string", "assertion failed"
    
try:
    pde = PDE(inputs=['a', 'b'], outputs=['b'])
except Exception as e:
    assert str(e) == "Repeated item b", "assertion failed"
    
try:
    pde = PDE(inputs=['a', 'b'], outputs=['c'], params=['a'])
except Exception as e:
    assert str(e) == "Repeated item a", "assertion failed"

## Summary

Get a summary of the PDE inputs, outputs, params and bocos.

In [ ]:
pde.summary()

inputs (train):  {'a': [], 'b': []}
inputs (test):  {'a': [], 'b': []}
outputs:  {'c': []}
params:  {'d': [], 'e': [], 'f': []}
bocos:  []



## Setting values

To solve a PDE you must set some input values, and optionally free-parameters. You cannot set output values (this will be given by the neural network).

In [ ]:
pde = PDE(inputs=['a', 'b'], outputs=['c'], params=['d'])

a = np.array([0, 0.5, 1])
b = np.array([0, 0.5, 1])
d = np.array([1.0])
pde.setValues({'a': a, 'b': b, 'd': d})

try:
    pde.setValues({'a': a, 'b': b, 'c': d})
except Exception as e:
    assert str(e) == "You cannot set values to outputs !", "assertion failed"
    
try:
    pde.setValues({'a': a, 'b': b, 'e': d})
except Exception as e:
    assert str(e) == "Key e not found !", "assertion failed"

By default, values are set for training but you can specify values for testing (in this case only for inputs).

In [ ]:
pde = PDE(inputs=['a', 'b'], outputs=['c'], params=['d'])

a = np.array([0, 0.5, 1])
b = np.array([0, 0.5, 1])
pde.setValues({'a': a, 'b': b}, train=False)

try:
    pde.setValues({'a': a, 'b': b, 'd': d}, train=False)
except Exception as e:
    assert str(e) == "You cannot set params in test data !", "assertion failed"

In [ ]:
#hide

pde = PDE(inputs=['a', 'b'], outputs=['c'], params=['d'])

a = np.array([0, 0.5, 1])
b = np.array([0, 0.5, 1])
d = np.array([1.0])

pde.setValues({'a': a})
pde.setValues({'a': a, 'b': b, 'd': d})


try:
    pde.setValues({'a': a, 'b': b, 'c': d})
except Exception as e:
    assert str(e) == "You cannot set values to outputs !", "assertion failed"
    
try:
    pde.setValues({'a': a, 'b': b, 'e': d})
except Exception as e:
    assert str(e) == "Key e not found !", "assertion failed"
    
pde.setValues({'a': a, 'b': b}, train=False)

try:
    pde.setValues({'a': a, 'b': b, 'd': d}, train=False)
except Exception as e:
    assert str(e) == "You cannot set params in test data !", "assertion failed"
    
try:
    pde.setValues({'a': a, 'b': b, 'c': d}, train=False)
except Exception as e:
    assert str(e) == "You cannot set values to outputs !", "assertion failed"
    
try:
    pde.setValues({'a': a, 'b': b, 'e': d}, train=False)
except Exception as e:
    assert str(e) == "Key e not found !", "assertion failed"

## Adding Boundary Conditions

To add a boundary condition to the system, first define one and then add it with the *addBoco* method.

In [ ]:
pde = PDE(inputs=['a', 'b'], outputs=['c'])

a1, a2 = np.array([0]), np.array([1])
b = np.array([1, 2, 3])

boco = PeriodicBoco({'a': a1, 'b': b}, {'a': a2, 'b': b})
pde.addBoco(boco)

pde.bocoSummary()

Periodic Boco Summary:
Input 1:  {'a': array([0]), 'b': array([1, 2, 3])}
Input 2:  {'a': array([1]), 'b': array([1, 2, 3])}

